In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# 한 페이지 내에 있는 제목과 링크를 한 번에 긁어오는 함수 생성
def get_paper_links(page_url):
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    doc_sum = soup.find_all('a', class_='docsum-title', href=True)
    
    titles = []
    links = []
    for doc in doc_sum:
        titles.append(doc.text.strip())
        links.append('https://pubmed.ncbi.nlm.nih.gov'+doc['href'])
    
    return titles, links

In [ ]:
# 모든 논문들의 제목과 링크 긁어오기

titles = []
links = [] 
for i in range(1, 44):
    
    # f string
    url = f'https://pubmed.ncbi.nlm.nih.gov/?term=korean%20journal%20of%20orthodontics&page={i}'
    
    # 위에서 생성한 함수 사용
    page_titles, page_links = get_paper_links(url)
    
    titles += page_titles
    links += page_links
    
    print(f'PAGE {i}/43 complete')

In [ ]:
# titles, links 하나의 칼럼으로 가지는 데이터프레임 생성
df = pd.DataFrame(data={'titles':titles, 'links':links})

In [ ]:
def crawling_doc(url):
    # url Html 구조 긁어오기
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # title 긁어오기
    article_page = soup.find('div', class_='article-page')
    article_details = article_page.find('main', class_='article-details')
    full_view = article_details.find('div', class_='full-view')
    title = full_view.find('h1', class_='heading-title')
    title = title.get_text(strip=True)
    
    # abstract 긁어오기
    abstract = soup.find('div', class_='abstract-content selected')
    
    if abstract:
        contents = abstract.find_all('p')
        abstracts = []
        for content in contents:
            all_txt = content.get_text(strip=True)
            abstracts.append(all_txt)
           
        abstracts =' '.join(abstracts)
    else:
        abstracts = 'Error'
         
    # meta_name 긁어오기
    div_authors = soup.find('div', class_='authors')
    if div_authors:
        authors = div_authors.find_all('a', class_='full-name')
    
        meta_name = [] 

        for author in authors:
            tmp = author.get_text(strip=True)
            meta_name.append(tmp)
    
        meta_name = ','.join(meta_name)
    
    else:
        meta_name = 'Error'
   
    # meta_keywords 긁어오기
    try:
        div_abstract = soup.find('div', class_='abstract')   
        div_abstract.find('div', class_='abstract-content selected').decompose()

        keywords = div_abstract.find('p').get_text(strip=True)
        keywords = keywords.split(':')[1]
        meta_keywords = keywords = keywords.split('; ')
        meta_keywords = ','.join(meta_keywords)
    except:
        meta_keywords = 'Error'

    return title, abstracts, meta_name, meta_keywords


In [ ]:
import time

abstracts = []
meta_names = []
meta_keywords = []

list_links = list(df.links)

for i, url in enumerate(list_links):
    print(f"{i+1} / {len(list_links)} COMPLETE")
    _, abstract, meta_name, meta_keyword = crawling_doc(url)
    
    abstracts.append(abstract)
    meta_names.append(meta_name)
    meta_keywords.append(meta_keyword)


In [ ]:
# 데이터프레임 생성
# 첫 번째 방법 : 새로 만들기
result_df = pd.DataFrame(data={'titles':titles,
                               'links':links,
                               'abstracts':abstracts,
                               'meta_names':meta_names,
                               'meta_keywords':meta_keywords})
result_df.head()

In [ ]:
!pwd

In [ ]:
# /Users/geoffrey/jupyter/lesson/201227/test_folder/crawling_result.csv
result_df.to_csv("./test_folder/crawling_result.csv", index=False)

In [ ]:
load_df = pd.read_csv("crawling_result.csv", index_col=0)

In [ ]:
load_df

In [ ]:
# 판다스의 구조 
# iloc, loc 차이점 반드시 숙지
# iloc : 행 기준 인덱싱
# loc : index 기준 인덱싱 -> 판다스의 구조 알고 있어야 함

In [ ]:
result_df.loc[0]

In [ ]:
tmp_dict = {}
tmp_dict['idx'] = [1, 3, 5, 7, 9]
tmp_dict['column'] = ['a', 'p', 'p', 'l', 'e']

In [ ]:
tmp_df = pd.DataFrame(tmp_dict)
tmp_df

In [ ]:
print(tmp_df.iloc[0])
print(tmp_df.loc[0])


In [ ]:
tmp_df = tmp_df.set_index('idx')
tmp_df

In [ ]:
tmp_df.iloc[0]

In [ ]:
tmp_df.loc[9]

In [ ]:
result_df.head()

In [ ]:
# 다음 진행 사항
# 1. 필요 없는 단어 제거
# 2. pos tagging

In [ ]:
# Objective: , Methods:, Results:, Conclusions:
test = result_df.abstracts[2]
test

In [ ]:
test_split = test.split(' ')
test_split

In [ ]:
# try, except로 : 기준으로 나눠보기
# 완성시켜보기
for x in test_split:
    try:
        tmp = x.split(':')
        tmp = x[1]
        
    except:
        pass

In [ ]:
# 특정 단어들이 해당 문자열에 포함되어 있는지 확인
check_words = ['Objective:', 'Methods:', 'Results:', 'Conclusions:']

In [ ]:
def check_token(token):
    for word in check_words:
        if word in token:
            return token.split(':')[1]
        else:
            continue
    return token

In [ ]:
test_token = 'Results:IMPORTANT!'
check_token(test_token)

In [ ]:
result_tokens = []
for token in test_split:
    result_tokens.append(check_token(token))

In [ ]:
result_tokens

In [ ]:
# *map function
# [A, B, C, D, ..., N]
# 리스트의 엘리먼트에 원하는 함수를 적용

# 기존 방법
result_tokens = []
for token in test_split:
    result_tokens.append(check_token(token))

# map function 활용
result_tokens = list(map(lambda x: check_token(x), test_split))

# 받는 인자가 하나인 경우 그냥 이런식으로 표현해도 됨!
# list(map(check_token, test_split))

# 예제 하나 더 : 모든 리스트 내부 엘리먼트들을 문자열 형태로 변환하기
# test = [1, 2, 3, 4, 5]
# list(map(str, test))

In [ ]:
print(result_tokens)

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# stop words 목록 확인
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words

In [ ]:
import os

os.mkdir